In [42]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from Storage.storage import get_cursor, get_info_for_song_id
from Fingerprint.fingerprint import fingerprint_file
from recognise import recognise_song

In [9]:
# Baseline accuracy

TEST_DIR = "test/ab/"

correct = 0

for song in os.listdir(TEST_DIR):
    result = recognise_song(TEST_DIR + song)
    result_title = result[2]
    if result_title.replace(" ", "").lower() in song:
        correct += 1

print(f"{correct}/{len(os.listdir(TEST_DIR))}")

10/23


In [74]:
def shazam_match(offsets, fname, supposed):
    tks = list(map(lambda x: x[0] - x[1], offsets))
    # number of matches in highest peak
    hist, _ = np.histogram(tks)
#     plt.plot(_[:-1], hist)
#     plt.title(fname)
#     plt.show()
    return np.max(hist)

In [73]:
def new_get_matches(hashes, threshold=5):
    conn, c = get_cursor()
    h_dict = {}
    for h, t, _ in hashes:
        h_dict[h] = t
    in_values = f"({','.join([str(h[0]) for h in hashes])})"
    c.execute(f"SELECT hash, offset, song_id FROM hash WHERE hash IN {in_values}")
    results = c.fetchall()
    result_dict = defaultdict(list)
    for r in results:
        result_dict[r[2]].append((r[1], h_dict[r[0]]))
    return result_dict

def new_recognise_song(filename):
    hashes = fingerprint_file(filename)
    matches = new_get_matches(hashes)
    matched_song = None
    best_score = 0
    print(f"Recognising {filename} ---")
    for song_id, offsets in matches.items():
        score = shazam_match(offsets, song_id, filename.split("/")[-1])
#         print(f"{song_id} - {score}")
        if score > best_score:
            best_score = score
            matched_song = song_id
    info = get_info_for_song_id(matched_song)
    print(best_score)
    if info is not None:
        return info
    return matched_song

In [78]:
correct = 0

for song in os.listdir(TEST_DIR):
    result = new_recognise_song(TEST_DIR + song)
    result_title = result[2]
    if result_title.replace(" ", "").lower() in song:
        correct += 1
    else:
        print(f"{song} - {result_title}")
    print("--------")

print(f"{correct}/{len(os.listdir(TEST_DIR))}")

Recognising test/ab/openyoureyes1.wav ---
6
--------
Recognising test/ab/allhopeisgone1.wav ---
9
--------
Recognising test/ab/stillremains2.wav ---
7
stillremains2.wav - Wayward One
--------
Recognising test/ab/burnitdown2.wav ---
28
--------
Recognising test/ab/openyoureyes2.wav ---
9
--------
Recognising test/ab/brokenwings2.wav ---
44
--------
Recognising test/ab/brokenwings1.wav ---
58
--------
Recognising test/ab/onedayremains3(quiet).wav ---
8
onedayremains3(quiet).wav - Buried Alive
--------
Recognising test/ab/burnitdown1.wav ---
20
--------
Recognising test/ab/stillremains3.wav ---
4
stillremains3.wav - Broken Wings
--------
Recognising test/ab/wonderfullife2.wav ---
14
--------
Recognising test/ab/findthereal1.wav ---
6
findthereal1.wav - Broken Wings
--------
Recognising test/ab/onedayremains1.wav ---
16
--------
Recognising test/ab/makeitright1.wav ---
8
--------
Recognising test/ab/makeitright3.wav ---
7
makeitright3.wav - Peace Is Broken
--------
Recognising test/ab/find

In [76]:
wrong_songs = [
    "stillremains2.wav",
    "onedayremains3(quiet).wav",
    "stillremains3.wav",
    "findthereal1.wav",
    "makeitright3.wav",
    "findthereal3.wav",
    "onedayremains2(loud).wav",
    "makeitright2.wav",
    "stillremains1.wav",
]

for song in wrong_songs:
    result = new_recognise_song(TEST_DIR + song)

Recognising test/ab/stillremains2.wav ---
7
Recognising test/ab/onedayremains3(quiet).wav ---
8
Recognising test/ab/stillremains3.wav ---
4
Recognising test/ab/findthereal1.wav ---
6
Recognising test/ab/makeitright3.wav ---
7
Recognising test/ab/findthereal3.wav ---
10
Recognising test/ab/onedayremains2(loud).wav ---
8
Recognising test/ab/makeitright2.wav ---
10
Recognising test/ab/stillremains1.wav ---
4
